#Lista 2
-------------


Objetivo: treinar um modelo de classificação utilizando o modelo random Forest e XGBoost.


Banco utilizado: Detecção de doenças no fígado.


[Link](https://drive.google.com/file/d/1jnLwuv4e_ZeCqluMXs3Mar2TGetumtVv/view?usp=drive_link) para o banco


Descrição do banco:
> Este banco contém registro de 416 pacientes diagnosticados com doença no fígado e 167 pacientes sem. Esta informação está na coluna Selector.
>
> Há 10 variáveis no banco:
>
> * age: idade
> * Gender: gênero do paciente
> * TB: Bilirrubina total
> * DB: Bilirrubina direta
> * Alkphos: fosfatase alcalina.
> * Sgpt: transaminase glutâmico-pirúvica sérica (TGP)
> * Sgot: transaminase glutâmico-oxalacética sérica (TGO)
> * TP: Proteína total
> * ALB: Albumina
> * A/G Ratio: Relação Albumina:Globulina


Exercício:


1. Carregue o banco de dados e analise suas features. Transforme a feature sexo em uma variavel dummy.
2. Separe o banco em 80% para treino e 20% para teste.
3. Treine um modelo de Random Forest com parâmetros de n_estimators de 150 e max_depth de 5.
  3.1 Quais são as duas features mais importantes?
4. Treine um modelo de XGboost com parâmetros de max_depth de 5, learning rate de 0.1 e n_estimators de 150.
  4.1 Quais são as duas features mais importantes?


5. Construa um report dos dois modelos.
  5.1 Tomando como decisão a precisão para detectar doença no fígado qual é o melhor modelo?
 5.2 Tomando como decisão o f1 score do rótulo de pacientes com doença no fígado qual é o melhor modelo?


Obs.: Utilize o **mesmo** banco de treino e de teste para construir e avaliar os dois modelos.



##1. Carga do banco
--------------------

In [1]:
import pandas as pd

# Carregar o banco de dados
df = pd.read_csv("Indian Liver Patient Dataset (ILPD).csv")

# Analisar as features
print(df.head())

   Age  Gender    TB   DB  Alkphos  Sgpt  Sgot   TP  ALB  A/G Ratio  Selector
0   65  Female   0.7  0.1      187    16    18  6.8  3.3       0.90         1
1   62    Male  10.9  5.5      699    64   100  7.5  3.2       0.74         1
2   62    Male   7.3  4.1      490    60    68  7.0  3.3       0.89         1
3   58    Male   1.0  0.4      182    14    20  6.8  3.4       1.00         1
4   72    Male   3.9  2.0      195    27    59  7.3  2.4       0.40         1


In [2]:
# Transformar a feature sexo em uma variável dummy
df = pd.get_dummies(df, columns=['Gender'], drop_first=True)

# transformando selector em 0 e 1   
df['Selector'] = df['Selector'].map({1:0, 2:1})

# Tratar valores ausentes
df.fillna(df.mean(), inplace=True)

In [3]:
df.dtypes

Age              int64
TB             float64
DB             float64
Alkphos          int64
Sgpt             int64
Sgot             int64
TP             float64
ALB            float64
A/G Ratio      float64
Selector         int64
Gender_Male      uint8
dtype: object

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,583.0,44.746141,16.189833,4.0,33.0,45.000000,58.0,90.0
TB,583.0,3.298799,6.209522,0.4,0.8,1.000000,2.6,75.0
DB,583.0,1.486106,2.808498,0.1,0.2,0.300000,1.3,19.7
Alkphos,583.0,290.576329,242.937989,63.0,175.5,208.000000,298.0,2110.0
Sgpt,583.0,80.713551,182.620356,10.0,23.0,35.000000,60.5,2000.0
Sgot,583.0,109.910806,288.918529,10.0,25.0,42.000000,87.0,4929.0
TP,583.0,6.483190,1.085451,2.7,5.8,6.600000,7.2,9.6
ALB,583.0,3.141852,0.795519,0.9,2.6,3.100000,3.8,5.5
A/G Ratio,583.0,0.947064,0.318492,0.3,0.7,0.947064,1.1,2.8
Selector,583.0,0.286449,0.452490,0.0,0.0,0.000000,1.0,1.0


##2. Quebra do banco
--------------------

In [5]:
from sklearn.model_selection import train_test_split

# Separar o banco em 80% para treino e 20% para teste
X = df.drop('Selector', axis=1)
Y = df['Selector']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



##3. Primeiro modelo
--------------------

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Treinar o modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=150, max_depth=5, random_state=42)
rf_model.fit(X_train, Y_train)

# Interpretar as duas features mais importantes
feature_importances_rf = rf_model.feature_importances_
feature_names = X_train.columns
top_features_rf = sorted(zip(feature_importances_rf, feature_names), reverse=True)[:2]
print("As duas features mais importantes para o modelo de Random Forest são:")
for importance, feature in top_features_rf:
    print(f"{feature}: {importance}")


As duas features mais importantes para o modelo de Random Forest são:
Alkphos: 0.17033439155633562
DB: 0.1448663462735451


##3.1 Interpretação de dois coeficientes
--------------------

A impureza (ou pureza) de um nó é medida usando métricas como o índice Gini ou entropia para problemas de classificação. O total da redução da impureza para cada característica é então normalizado pelo total da redução da impureza de todas as características, resultando em um valor entre 0 e 1. Um valor de importância mais alto (próximo de 1) indica que a característica é mais importante para as previsões do modelo. Neste caso as duas características mais importantes para o modelo Random Forest são "Alkphos" e "Direta Bilirrubina".

"Alkphos" pode se referir a uma medida de fosfatase alcalina no sangue, que é um enzima relacionada a várias funções do corpo, incluindo a saúde óssea e hepática. A fosfatase alcalina é uma enzima que é produzida principalmente pelo fígado e ossos. A quantidade de fosfatase alcalina no sangue aumenta quando o fígado está danificado ou quando há um bloqueio no fluxo da bile. Portanto, faz sentido que a característica "Alkphos" seja uma das mais importantes para prever a presença de doença hepática.

"Direta Bilirrubina" é um tipo de bilirrubina que é conjugada com ácido glicurônico no fígado e é excretada na bile. A bilirrubina é um produto da degradação da hemoglobina e é normalmente processada pelo fígado. A presença de bilirrubina direta no sangue pode ser um sinal de problemas hepáticos, como hepatite ou cirrose. Portanto, a característica "Direta Bilirrubina" também é uma variável importante para prever a presença de doença hepática.

##4. Segundo modelo
--------------------

In [7]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=150, random_state=42)
model_xgb.fit(X_train, Y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=150, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

##4.1 Duas features mais importantes
--------------------

In [8]:
pd.DataFrame(model_xgb.feature_importances_, index=X_train.columns, columns=['importance']).sort_values(by='importance', ascending=False).head(2)

,importance
DB,0.224398
TB,0.127188


##5. Avaliação do modelo
--------------------

In [9]:
# Construir um report dos dois modelos
from sklearn.metrics import classification_report
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = model_xgb.predict(X_test)
print("Relatório de classificação para o modelo de Random Forest:")
print(classification_report(Y_test, y_pred_rf))
print("Relatório de classificação para o modelo de XGBoost:")
print(classification_report(Y_test, y_pred_xgb))

Relatório de classificação para o modelo de Random Forest:
              precision    recall  f1-score   support

           0       0.77      0.91      0.84        87
           1       0.47      0.23      0.31        30

    accuracy                           0.74       117
   macro avg       0.62      0.57      0.57       117
weighted avg       0.70      0.74      0.70       117

Relatório de classificação para o modelo de XGBoost:
              precision    recall  f1-score   support

           0       0.80      0.84      0.82        87
           1       0.46      0.40      0.43        30

    accuracy                           0.73       117
   macro avg       0.63      0.62      0.62       117
weighted avg       0.71      0.73      0.72       117


##5.1 Qual modelo escolhido olhando para a precisão de detecção de doença no fígado?
--------------------

Comparando os relatórios dos dois modelos:
- O modelo de Random Forest tem uma precisão de 0.47 para detectar doença no fígado.
- O modelo de XGBoost tem uma precisão de 0.46 para detectar doença no fígado.

##5.2 Qual modelo escolhido olhando para o f1-score do rótulo de doença do fígado?
--------------------

Comparando os relatórios dos dois modelos:
- O modelo de Random Forest tem um f1-score de 0.31 para o rótulo de pacientes com doença no fígado.
- O modelo de XGBoost tem um f1-score de 0.43 para o rótulo de pacientes com doença no fígado.

Os dois modelos foram bem insatisfatórios, mas o modelo de XGBoost teve um desempenho ligeiramente melhor em termos de f1-score para o rótulo de pacientes com doença no fígado. Portanto, com base nesse critério, o modelo de XGBoost seria a escolha preferida. No entanto, ambos os modelos têm uma precisão muito baixa para detectar doença no fígado, o que sugere que eles podem não ser adequados para este problema específico. Como os estimadores de Random Forest e XGBoost são sensíveis a hiperparâmetros, pode ser necessário ajustar os hiperparâmetros como learning rate, max_depth, n_estimators, e limiar de probabilidade para melhorar o desempenho dos modelos.


# Repository of all projects 
a repository of all project 